[All About Strictness - 12 Sep 2017 Michael Snoyman](https://www.fpcomplete.com/blog/2017/09/all-about-strictness)

# Bang!

how we can force Haskell to be more strict in its evaluation. 

In [1]:
{-# LANGUAGE BangPatterns #-}
add :: Int -> Int -> Int
add !x !y = x + y

main :: IO ()
main = do
  let !five = add (1 + 1) (1 + 2)
      !seven = add (1 + 2) (1 + 3) -- not used but evaluated immediately, no thunk here

  putStrLn $ "Five: " ++ show five
main

Five: 5

bang patterns are just **syntactic sugar** for something else. And in this case, that something else is the ***seq function***. This function looks like:

`seq :: a -> b -> b`

In [2]:
:t seq

seq :: forall b a. a -> b -> b

we could implement this type signature yourself, of course, by just ignoring the a value:

In [3]:
badseq :: a -> b -> b
badseq a b = b

However, seq **uses primitive operations from GHC itself** to ensure that, ***when b is evaluated, a is evaluated too***.

In [4]:
add :: Int -> Int -> Int
add x y =
  let part1 = seq x part2
      part2 = seq y answer
      answer = x + y
   in part1   -- m: part1 refers transitively to answer
-- Or more idiomatically
add x y = x `seq` y `seq` x + y

add 3 7

10

In [5]:
main :: IO ()
main = do
  let five = add (1 + 1) (1 + 2)
      seven = add (1 + 2) (1 + 3)

  five `seq` seven `seq` putStrLn ("Five: " ++ show five) -- ? m: evaluation of putStrLn is not forced
main  

Five: 5

# Tracing evaluation

In [6]:
import Debug.Trace

add :: Int -> Int -> Int
add x y = x + y

main :: IO ()
main = do
  let five = trace "five" (add (1 + 1) (1 + 2))
      seven = trace "seven" (add (1 + 2) (1 + 3))

  putStrLn $ "Five: " ++ show five
--Five: five
--5


In [7]:
{-# LANGUAGE BangPatterns #-}
import Debug.Trace

add :: Int -> Int -> Int
add x y = x + y

main :: IO ()
main = do
  let !five = trace "five" (add (1 + 1) (1 + 2))
      !seven = trace "seven" (add (1 + 2) (1 + 3))

  putStrLn $ "Five: " ++ show five
  
  five `seq` seven `seq` putStrLn ("Five: " ++ show five)
  
--seven
--five
--Five: 5
--Five: 5

the order of their output may be different than you expect. On my system, for example, seven prints before five. That's because GHC retains the right to rearrange order of evaluation in these cases.

By contrast, if you use five `seq` seven `seq` putStrLn ("Five: " ++ show five), it will (should?) **always come out in the same order**: first five, then seven, then "Five: 5". This gives **a bit of a lie** to my claim that bang patterns are always a **simple translation to seqs**. 

However, as long as your expressions are **truly pure**, you will be **unable to observe the difference** between the two.

In [8]:
{-# LANGUAGE BangPatterns #-}
import Debug.Trace

add :: Int -> Int -> Int
add x y = x + y

main :: IO ()
main = do
  let five = trace "five" (add (1 + 1) (1 + 2))
  let seven = trace "seven" (add (1 + 2) (1 + 3))
  five `seq` seven `seq` putStrLn ("Five: " ++ show five)
--five
--seven
--Five: 5

1. Haskell is lazy by default
1. You can use bang patterns and seq to make things strict
1. By contrast, in strict languages, you can **use closures to make things lazy**
1. You can **see if a function is strict** in its **arguments** by **passing in bottom (undefined)** and seeing if it explodes in your face
1. The **trace function** can help you see this as well

### implement an average function

use a helper datatype, calling RunningTotal, to capture both the cumulative sum and the number of elements we've seen so far

In [9]:
data RunningTotal = RunningTotal
  { sum :: Int
  , count :: Int
  }

printAverage :: RunningTotal -> IO ()
printAverage (RunningTotal sum count)
  | count == 0 = error "Need at least one value!"
  | otherwise = print (fromIntegral sum / fromIntegral count :: Double)

-- | A fold would be nicer... we'll see that later
printListAverage :: [Int] -> IO ()
printListAverage =
  go (RunningTotal 0 0)
  where
    go rt [] = printAverage rt
    go (RunningTotal sum count) (x:xs) =
      let rt = RunningTotal (sum + x) (count + 1)
       in go rt xs

main :: IO ()
main = printListAverage [1..1000000]

main

500000.5

We're going to run this with run time statistics turned on so we can look at memory usage:

`$ stack ghc average.hs && ./average +RTS -s `

Lo and behold, our memory usage is through the roof! Probably we should force evaluation of the newly constructed `rt` before recursing back into `go`. Unfortunately, this results in exactly the same memory usage as we had before!???

```
[1 of 1] Compiling Main             ( average.hs, average.o )
  Linking average ...
  500000.5
     258,654,528 bytes allocated in the heap
     339,889,944 bytes copied during GC
      95,096,512 bytes maximum residency (9 sample(s))
       1,148,312 bytes maximum slop
             164 MB total memory in use (0 MB lost due to fragmentation)
             ```

## Weak Head Normal Form
[there's a great Stack Overflow answer](https://stackoverflow.com/a/6889335/369198)

In [10]:
main = putStrLn $ undefined `seq` "Hello World"
main

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:19 in interactive:Ghci166

 t will print an error about undefined, since it will try to evaluate undefined before it will evaluate "Hello World", and because **putStrLn** is ***strict in its argument***.

In [11]:
main = putStrLn $ Just undefined `seq` "Hello World"
main

Line 1: Redundant seq
Found:
Just undefined `seq` "Hello World"
Why not:
"Hello World"

Hello World

It turns out that when we talk about forcing evaluation with **seq**, we're only talking about **evaluating to weak head normal form (WHNF)**. For most data types, this means ***unwrapping one layer of constructor***.

`Just undefined`

In the case of `Just undefined`, it means that we unwrap the Just data constructor, but **don't touch the `undefined`** within it. 

with a standard data constructor?, the impact of using seq is the same as ***pattern matching the outermost constructor***.

If you want to **monomorphise**, for example, you can implement a function of type 

`seqMaybe :: Maybe a -> b -> b` 

and use it in the main example above.

In [12]:
seqMaybe :: Maybe a -> b -> b
seqMaybe Nothing b = b
seqMaybe (Just _) b = b

main :: IO ()
main = do
  putStrLn $ Just undefined `seqMaybe` "Hello World"
  --putStrLn $ undefined `seqMaybe` "Goodbye!"
  
  putStrLn $ error `seq` "Hello" -- ! error without arguments:  any function applied to too few values is automatically in WHNF.
  putStrLn $ (\x -> undefined) `seq` "World" -- a function fully applied to its arguments. It's no longer a function, it's a value. 
  --putStrLn $ error "foo" `seq` "Goodbye!"  
  --putStrLn $ undefined 5 `seq` "Hello" 
  --putStrLn $ (error "foo" :: Int -> Double) `seq` "Hello" 
  --putStrLn $ (error "foo" :: Void -> ()) `seq` "Hello" 
  
data Void  
:t undefined 5  
:t error
main  

undefined 5 :: forall t. t

error :: forall a. [Char] -> a

Hello World
Hello
World

Instead of putting the bangs on the RunningTotal value, I'm putting them **on the values within the constructor**, forcing them to be evaluated at each loop

We're no longer accumulating a huge chain of thunks, and our maximum residency drops to 44kb. (Total allocations, though, are still up around 192mb. 
We need to play around with other optimizations outside the scope of this post to deal with the total allocations, so we're going to ignore this value for the rest of the examples

In [13]:
go rt [] = printAverage rt
go (RunningTotal !sum !count) (x:xs) =   -- <=== !sum !count
  let rt = RunningTotal (sum + x) (count + 1)
   in go rt xs

Alternative approach:

**forces evaluation** of the new sum and count **before constructing the new RunningTotal value**. I like this version a bit more, as it's forcing evaluation at the correct point: when creating the value, ***instead of on the next iteration of the loop when destructing it***.

Moral of the story: make sure you're evaluating the thing you actually need to evaluate, ***not just its container***!

In [14]:
-- alternative
go rt [] = printAverage rt
go (RunningTotal sum count) (x:xs) =
  let !sum' = sum + x
      !count' = count + 1
      rt = RunningTotal sum' count'
   in go rt xs

## deepseq

fully evaluate down to normal form (NF), meaning all thunks have been evaluated inside our values. 

semi-standard (meaning it ships with GHC) library to handle this: deepseq. It works by providing an **`NFData`** type class the defines how to reduce a value to normal form (via the **`rnf`** method).

In [15]:
import Control.DeepSeq

instance NFData RunningTotal where
  rnf (RunningTotal sum count) = sum `deepseq` count `deepseq` ()
  
printListAverage :: [Int] -> IO ()
printListAverage =
  go (RunningTotal 0 0)
  where
    go rt [] = printAverage rt
    go (RunningTotal sum count) (x:xs) =
      let rt = RunningTotal (sum + x) (count + 1)
       in rt `deepseq` go rt xs  -- <=================

In [17]:
-- or with default implementation
{-# LANGUAGE DeriveGeneric #-}
import GHC.Generics (Generic)

data RunningTotal = RunningTotal
  { sum :: Int
  , count :: Int
  }
  deriving Generic
instance NFData RunningTotal

We can use this not only to avoid space leaks (as we're doing here), but also **to avoid accidentally including exceptions inside thunks within a value**. For an example of that, check out the tryAnyDeep function from the safe-exceptions library.

## Strict DATA (not args or functions!)
### strictness annotations

because of laziness, what we're actually saying is that a RunningTotal value could **contain two Ints**, or it could **contain thunks** that will evaluate to Ints, or thunks that will throw exceptions.

Instead, we'd like to make it impossible to construct a RunningTotal value that has **any laziness room left over**. And to do that, we can use strictness annotations in our definition of the data type:

In [18]:
data RunningTotal = RunningTotal
  { sum :: !Int
  , count :: !Int
  }
  deriving Generic

Whenever you **evaluate a value of type RunningTotal**, you **must also evaluate the two Ints it contains**

As we mentioned above, our second go clause forces evaluation of the RunningTotal value by **taking apart its constructor**. This act **now automatically forces evaluation of sum and count**, which we previously needed to achieve via a bang pattern.

Additional Bonus:

When dealing with **small values like an Int**, GHC will automatically **unbox strict fields**. This means that, instead of keeping a pointer to an Int inside RunningTotal, it will keep the Int itself.

### "how do I know if I should use a strictness annotation on my data fields?" 

unless you know that you want laziness for a field, **make it strict**. 

Making your fields strict helps in a few ways:
1. Avoids accidental space leaks, like we're doing here
1. Avoids **accidentally including bottom values** TODO ?
1. When constructing a value with record syntax, GHC will give you an error if you forget a strict field. It will only give you a warning for non-strict fields.

## The curious case of newtype

In [56]:
data Foo = Foo Int -- boxed with thunks
data Bar = Bar !Int -- still boxed but without thunks
newtype Baz = Baz Int -- unboxed

case undefined of { Foo _ -> putStrLn "Still alive!" }

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:1 in interactive:Ghci896

In [30]:
case Foo undefined of { Foo _ -> putStrLn "Still alive!" }

Still alive!

In [31]:
case undefined of { Bar _ -> putStrLn "Still alive!" }

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:1 in interactive:Ghci466

In [33]:
case Bar undefined of { Bar _ -> putStrLn "Still alive!" }

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:10 in interactive:Ghci488

### What we know about newtypes is that they have no runtime representation. 

Therefore, it's impossible for the Baz data constructor to be hiding an extra layer of bottomness. In other words, **Baz undefined and undefined are indistinguishable**. That may sound like Bar at first, but interestingly it's not.

In [36]:
case undefined of { Baz _ -> putStrLn "Still alive!" }
case undefined of { Baz x -> putStrLn "Still alive!" }

Still alive!

Still alive!

In [34]:
case Baz undefined of { Baz _ -> putStrLn "Still alive!" }

Still alive!

unwrapping a Baz constructor can have **no effect on runtime behavior**, since it **was never there in the first place**. The pattern match inside (5), therefore, does nothing. It is equivalent to 

In [37]:
case undefined of { _ -> putStrLn "Still alive!" }

Line 1: Redundant case
Found:
case undefined of
    _ -> putStrLn "Still alive!"
Why not:
putStrLn "Still alive!"

Still alive!

In [45]:
case undefined of { x -> putStrLn "Still alive!" } 

Still alive!

In [47]:
case undefined of { !x -> putStrLn "Still alive!" } 

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:1 in interactive:Ghci795

In [49]:
case undefined of { x -> putStrLn ("Still alive!" ++ show x)} 

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:1 in interactive:Ghci817

In [53]:
Baz undefined `seq` putStrLn "Still alive!" -- newtype is not boxed

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:5 in interactive:Ghci861

In [57]:
Baz undefined

<interactive>:1:1: error:
    • No instance for (Show Baz) arising from a use of ‘print’
    • In a stmt of an interactive GHCi command: print it

### Convenience operators and functions

Bang patterns help, but there are other ways to force evaluation. Perhaps the most common is the `$!` operator (uses `seq`), e.g.:

In [59]:
mysum :: [Int] -> Int
mysum list0 =
  go list0 0
  where
    go [] total = total
    go (x:xs) total = go xs $! total + x  --     <====================    

main = print $ mysum [1..1000000]

This **forces evaluation of total + x** before recursing back into the go function, avoiding a space leak. (EXERCISE: do the same thing with a bang pattern, and with the seq function.)

The `$!!` operator is the same, except instead of working with seq, it **uses deepseq** and therefore **evaluates to normal form**.



In [61]:
import Control.DeepSeq

average :: [Int] -> Double
average list0 =
  go list0 (0, 0)
  where
    go [] (total, count) = fromIntegral total / count
    go (x:xs) (total, count) = go xs $!! (total + x, count + 1) -- <==== we can use complex exprs with (+)

main = print $ average [1..1000000]

Another nice helper function is `force`. What this does is makes it that, when the expression you're looking at is evaluated to WHNF, it's actually evaluated to **NF**. For example, we can rewrite the go function above as:

In [63]:
go [] (total, count) = fromIntegral total / count
go (x:xs) (total, count) = go xs $! force (total + x, count + 1) --   <==== force

## Data structures

In [79]:
data List a = Cons a (List a) | Nil

main = Cons undefined undefined `seq` putStrLn "Hello World"
main

:t Cons undefined 
:t Cons undefined undefined -- why undefined is accepted ? undefined can be `List a` !!!
--:t Cons undefined $ Cons undefined -- rejected
:t Cons undefined $ Cons undefined Nil

Hello World

Cons undefined :: forall a. List a -> List a

Cons undefined undefined :: forall a. List a

Cons undefined $ Cons undefined Nil :: forall a. List a

`Cons undefined undefined` is already in WHNF, since we've got the outermost constructor available

realize that Cons is the same as the : data constructor for lists, and see that the above is identical to:

In [74]:
main = (undefined:undefined) `seq` putStrLn "Hello World"
main

Hello World

This tells me that lists are a lazy data structure: I have a **bottom value for the first element**, a bottom value for the rest of the list, and yet **this first cell [x:xs] is not bottom**.

In [76]:
data List a = Cons a !(List a) | Nil

main = Cons undefined undefined `seq` putStrLn "Hello World"
main

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:2:23 in interactive:Ghci1266

This is going to explode in our faces! We are now strict in the tail of the list.

In [84]:
data List a = Cons a !(List a) | Nil -- strictness on tail constructor but not on values

main = Cons undefined (Cons undefined Nil) `seq` putStrLn "Hello World"
main

Hello World

With this definition of a list, we **need to know all the details about the list itself**, but the values **can remain undefined**. This is called ***spine strict***. By contrast, we can **also be strict in the values** and be ***value strict***:

In [85]:
data List a = Cons !a !(List a) | Nil

main = Cons undefined (Cons undefined Nil) `seq` putStrLn "Hello World"
main

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:2:13 in interactive:Ghci1428

There's one final definition of list you may be expecting, one strict in values but not in the tail:

In [87]:
data List a = Cons !a (List a) | Nil

In practice, I'm aware of no data structures in Haskell that follow this pattern, and therefore it doesn't have a name. 

### Vectors
The vectors in Data.Vector (also known as boxed vectors) are spine strict.

In [96]:
import qualified Data.Vector as V

main = V.fromList [undefined,undefined] `seq` putStrLn "Hello World"
main

Hello World

In [97]:
main = V.fromList (undefined:undefined) `seq` putStrLn "Hello World" 
main
-- !!!  fails, since the spine of the tail of the list is undefined
-- we must destruct undefined without looking into values a

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:30 in interactive:Ghci1614

In [98]:
main = V.fromList undefined `seq` putStrLn "Hello World"
main

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:19 in interactive:Ghci1626

Now let's look at **unboxed vectors**. Because of inference issues, we need to help out GHC a little bit more. So starting with this head of a program:

In [102]:
import qualified Data.Vector.Unboxed as UV

fromList :: [Int] -> UV.Vector Int
fromList = UV.fromList

main = fromList [undefined] `seq` putStrLn "Hello World"
main

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:3:18 in interactive:Ghci1664

In [104]:
main = fromList (undefined:undefined) `seq` putStrLn "Hello World"
main

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:18 in interactive:Ghci1688

In [106]:
main = fromList undefined `seq` putStrLn "Hello World"
main

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:1:17 in interactive:Ghci1712

Unfortunately, to my knowledge, **there is no definition of a strict, boxed vector** in a public library. Such a data type would be useful **to help avoid space leaks** (such as the original question that triggered this blog post).

## Sets and Maps
If you look at the containers and unordered-containers packages, you may have noticed that the Map-like modules come in Strict and Lazy variants (e.g., `Data.HashMap.Strict` and `Data.HashMap.Lazy`) while the Set-like modules do not (e.g., `Data.IntSet`). This is because **all of these containers are spine strict**, and **therefore** must be **strict in the keys**. Since a set only has keys, no separate values, it must also be value strict.

A map, by contrast, has both keys and values. The lazy variants of the map-like modules are spine-strict, value-lazy, whereas the strict variants are both spine and value strict.

EXERCISE Analyze the Data.Sequence.Seq data type and classify it as either lazy, spine strict, or value strict.

## Function arguments
A function is considered strict in one of its arguments if, when the function is applied to a bottom value for that argument, the result is bottom. 

+ for **Int is strict in both of its arguments**, since: undefined + x is bottom, and x + undefined is bottom.

The : data constructor for lists is lazy in both its first and second argument. But if you have `data List = Cons !a !(List a) | Nil`, `Cons` is strict in both its first and second argument.

## Folds
A common place to end up getting tripped up by laziness is dealing with folds. The most infamous example is the foldl function, which lulls you into a false sense of safety only to dash your hopes and destroy your dreams:

In [109]:
mysum :: [Int] -> Int
mysum = foldl (+) 0

main :: IO ()
main = print $ mysum [1..1000000]
main

500000500000

This is so close to correct, and yet **uses 53mb of resident memory**! The solution is but a tick away, using the strict left fold foldl' function:
(Why does the Prelude expose a function (foldl) which is almost always the wrong one to use?)

In [111]:
import Data.List (foldl')

mysum :: [Int] -> Int
mysum = foldl' (+) 0 --   <==== foldl' is a strict version of foldl

main :: IO ()
main = print $ mysum [1..1000000]
main

500000500000

But the important thing to note about almost all functions that claim to be strict is that they are only strict to weak head normal form.

 frustrated that there is **no foldl' that evaluates to normal form**?

by just throwing in **a call to force**, you can easily upgrade a WHNF fold into a NF fold:

In [113]:
import Data.List (foldl')
import Control.DeepSeq (force)

average :: [Int] -> Double
average =
  divide . foldl' add (0, 0)
  where
    divide (total, count) = fromIntegral total / count
    add (total, count) x = force (total + x, count + 1)

main :: IO ()
main = print $ average [1..1000000]
main

500000.5

Like a good plumber, force patches that leak right up!!!

## Streaming data
One of the claims of streaming data libraries (like conduit) is that they promote constant memory usage. This may make you think that you can get away without worrying about space leaks. However, all of the comments about WHNF vs NF mentioned above apply. To prove the point, let's do average badly with conduit:

In [117]:
import Conduit

average :: Monad m => ConduitM Int o m Double
average =
  divide <$> foldlC add (0, 0)
  where
    divide (total, count) = fromIntegral total / count
    add (total, count) x = (total + x, count + 1)

main :: IO ()
main = print $ runConduitPure $ enumFromToC 1 1000000 .| average
main

500000.5

EXERCISE Make this program run in constant resident memory, by using:

1. The force function
1. Bang patterns
1. A custom data type with strict fields

In [119]:
:t force

force :: forall a. NFData a => a -> a

## Chain reaction
Look at this super strict program. It's got a special value-strict list data type. I've liberally sprinkled bang patterns and calls to seq throughout. I've used $!. How much memory do you think it uses?

In [122]:
--#!/usr/bin/env stack
-- stack --resolver lts-9.4 script
{-# LANGUAGE BangPatterns #-}

data StrictList a = Cons !a !(StrictList a) | Nil

strictMap :: (a -> b) -> StrictList a -> StrictList b
strictMap _ Nil = Nil
strictMap f (Cons a list) =
  let !b = f a
      !list' = strictMap f list
   in b `seq` list' `seq` Cons b list'

strictEnum :: Int -> Int -> StrictList Int
strictEnum low high =
  go low
  where
    go !x
      | x == high = Cons x Nil
      | otherwise = Cons x (go $! x + 1)

double :: Int -> Int
double !x = x * 2

evens :: StrictList Int
evens = strictMap double $! strictEnum 1 1000000

main :: IO ()
main = do
  let string = "Hello World"
      string' = evens `seq` string
  putStrLn string
main

Hello World

When executing our program, GHC is only interested in performing the IO actions it is told to perform by the main function. And main only says something about putStrLn string.

EXERCISES

1. Change putStrLn string to putStrLn string' and see what happens to memory usage. (Then undo that change for the other exercises.)
1. Use a bang pattern in main somewhere to get the great memory usage.
1. Add a seq somewhere in the putStrLn string line to force the greater memory usage.